In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="openai/gpt-oss-120b")

In [ ]:
from langgraph.graph import START, END, StateGraph

### Reducers
A reducer is a function which combines old state and new state

In [11]:
from typing import TypedDict, Annotated, List
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage,
    ToolMessage
)



In [5]:
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]


In [6]:
def system_node(state: AgentState):
    return {
        "messages": [
            SystemMessage(content="You are a helpful assistant.")
        ]
    }


In [7]:
def human_node(state: AgentState):
    return {
        "messages": [
            HumanMessage(content="Explain reducers in simple words.")
        ]
    }


In [8]:
def ai_node(state: AgentState):
    response = llm.invoke(state["messages"])
    return {
        "messages": [
            AIMessage(content=response.content)
        ]
    }

In [9]:
graph = StateGraph(AgentState)

graph.add_node("system", system_node)
graph.add_node("human", human_node)
graph.add_node("ai", ai_node)

graph.add_edge(START, "system")
graph.add_edge("system", "human")
graph.add_edge("human", "ai")
graph.add_edge("ai", END)

app = graph.compile()

In [12]:
result = app.invoke({"messages": []})

for msg in result["messages"]:
    print(type(msg).__name__, "->", msg.content)


SystemMessage -> You are a helpful assistant.
HumanMessage -> Explain reducers in simple words.
AIMessage -> ### What’s a “reducer”?

A **reducer** is just a function that takes some data, does something to it, and gives you back a new piece of data. The word comes from the idea of “reducing” a bunch of information down to a single result.

Think of it like a kitchen blender:

1. **You put ingredients in** (the current state and an “action” that tells the blender what to do).  
2. **You press the button** (the reducer runs).  
3. **Out comes a new smoothie** (the new state).  

The original ingredients stay unchanged – the blender gives you a fresh result instead of mutating the old ones.

---

## Where you usually see reducers

### 1. Redux (state management for JavaScript apps)

- **State**: the whole app’s data (e.g., a list of todos, the current user, UI flags).  
- **Action**: an object that says *what happened* (e.g., `{ type: "ADD_TODO", text: "Buy milk" }`).  
- **Reducer**: a 